# 📊 Проект: Рекомендательная система для социальной сети

## 🎯 Предсказание лайков пользователей
 
**Описание:** Обучение CatBoost модели для предсказания лайков постов пользователями на основе их демографических данных и контента постов.

## Импорт библиотек

In [1]:
import os
import time
import warnings
from io import StringIO
from pathlib import Path

import nltk
import numpy as np
import pandas as pd
import psycopg2
from catboost import CatBoostClassifier
from dotenv import load_dotenv
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.metrics import recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine
from sqlalchemy import text

from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import BertModel  # https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
from transformers import RobertaModel  # https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaModel
from transformers import DistilBertModel  # https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertModel

from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.cluster import KMeans

warnings.filterwarnings('ignore')

## 📥 2. Загрузка и исследование данных

### Описание датасетов:
- **user_data.csv**: Демографические данные пользователей (163K записей)
- **post_text.csv**: Тексты постов и их темы (7K записей)  
- **feed_data.csv**: Взаимодействия пользователей с постами (10M записей)

In [2]:
%%time

# Определяем пути
PROJECT_ROOT = Path().resolve().parent
RAW_DATA_DIR = PROJECT_ROOT / "part3-nlp-enhanced" / "raw_data"
PROCESSED_DATA_DIR = PROJECT_ROOT / "part3-nlp-enhanced" / "processed_data"

user_data = pd.read_csv(RAW_DATA_DIR / "user_data.csv")
post_text = pd.read_csv(RAW_DATA_DIR / "post_text.csv")
feed_data = pd.read_csv(RAW_DATA_DIR / "feed_data_10000000.csv")

CPU times: total: 15.4 s
Wall time: 16.5 s


In [3]:
%%time

display(user_data.info(),
        post_text.info(),
        feed_data.info())

display(user_data,
        post_text,
        feed_data)

<class 'pandas.DataFrame'>
RangeIndex: 163205 entries, 0 to 163204
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    163205 non-null  int64
 1   gender     163205 non-null  int64
 2   age        163205 non-null  int64
 3   country    163205 non-null  str  
 4   city       163205 non-null  str  
 5   exp_group  163205 non-null  int64
 6   os         163205 non-null  str  
 7   source     163205 non-null  str  
dtypes: int64(4), str(4)
memory usage: 10.0 MB
<class 'pandas.DataFrame'>
RangeIndex: 7023 entries, 0 to 7022
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   post_id  7023 non-null   int64
 1   text     7023 non-null   str  
 2   topic    7023 non-null   str  
dtypes: int64(1), str(2)
memory usage: 164.7 KB
<class 'pandas.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 5 columns):
 #   Column     Dtype
---  ------     -

None

None

None

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


,timestamp,user_id,post_id,action,target
0,2021-10-31 15:32:01,94388,3770,view,0
1,2021-11-01 10:12:08,94388,732,view,1
2,2021-11-01 10:13:31,94388,732,like,0
3,2021-11-01 10:13:33,94388,915,view,1
4,2021-11-01 10:14:55,94388,915,like,0
...,...,...,...,...,...
9999995,2021-11-12 06:58:12,54841,6482,view,0
9999996,2021-11-12 07:00:00,54841,2371,view,1
9999997,2021-11-12 07:02:54,54841,2371,like,0
9999998,2021-11-12 07:02:56,54841,982,view,0


CPU times: total: 141 ms
Wall time: 135 ms


## 🛠️ 3. Предобработка текстовых данных

In [4]:
%%time

def get_model(model_name):
    assert model_name in ['bert', 'roberta', 'distilbert']

    checkpoint_names = {
        'bert': 'bert-base-cased',  # https://huggingface.co/bert-base-cased
        'roberta': 'roberta-base',  # https://huggingface.co/roberta-base
        'distilbert': 'distilbert-base-cased'  # https://huggingface.co/distilbert-base-cased
    }

    model_classes = {
        'bert': BertModel,
        'roberta': RobertaModel,
        'distilbert': DistilBertModel
    }

    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(checkpoint_names[model_name])


tokenizer, model = get_model('distilbert')

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertModel LOAD REPORT from: distilbert-base-cased
Key                     | Status     |  | 
------------------------+------------+--+-
vocab_transform.weight  | UNEXPECTED |  | 
vocab_layer_norm.bias   | UNEXPECTED |  | 
vocab_transform.bias    | UNEXPECTED |  | 
vocab_layer_norm.weight | UNEXPECTED |  | 
vocab_projector.bias    | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


CPU times: total: 1.02 s
Wall time: 5.51 s


In [5]:
%%time

### Сделаем датасет для постов

class PostDataset(Dataset):
    def __init__(self, texts, tokenizer):
        super().__init__()
        self.texts = tokenizer(
            texts,
            add_special_tokens=True,
            return_token_type_ids=False,
            return_tensors='pt',
            truncation=True,
            padding=True
        )
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        return {
            'input_ids': self.texts['input_ids'][idx], 
            'attention_mask': self.texts['attention_mask'][idx]
        }

    def __len__(self):
        return len(self.texts['input_ids'])
        
dataset = PostDataset(post_text['text'].values.tolist(), tokenizer)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
loader = DataLoader(dataset, batch_size=32, collate_fn=data_collator, pin_memory=True, shuffle=False)


@torch.inference_mode()
def get_embeddings_labels(model, loader):
    model.eval()
    total_embeddings = []
    for batch in tqdm(loader):
        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}
        embeddings = model(**batch)['last_hidden_state'][:, 0, :]
        total_embeddings.append(embeddings.cpu())
    return torch.cat(total_embeddings, dim=0)


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cpu
CPU times: total: 17.4 s
Wall time: 6.52 s


In [6]:
%%time

embeddings = get_embeddings_labels(model, loader).numpy()
display(embeddings)

# Сохранение эмбеддингов в файл
np.save('embeddings.npy', embeddings)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 220/220 [32:43<00:00,  8.92s/it]


array([[ 3.63150924e-01,  4.89376448e-02, -2.64081329e-01, ...,
        -1.41593203e-01,  1.59179904e-02,  9.19513404e-05],
       [ 2.36416295e-01, -1.59501165e-01, -3.27798337e-01, ...,
        -2.89936006e-01,  1.19365111e-01, -1.62374461e-03],
       [ 3.75191450e-01, -1.13943860e-01, -2.40547046e-01, ...,
        -3.38919163e-01,  5.86938709e-02, -2.12656371e-02],
       ...,
       [ 3.40382367e-01,  6.64923042e-02, -1.63184434e-01, ...,
        -8.65626633e-02,  2.03403741e-01,  3.20906676e-02],
       [ 4.32092339e-01,  1.10913627e-02, -1.17306165e-01, ...,
         7.54014403e-02,  1.02739736e-01,  1.52742602e-02],
       [ 3.04277569e-01, -7.62157440e-02, -6.77585676e-02, ...,
        -5.43487109e-02,  2.44383678e-01, -1.41484402e-02]],
      shape=(7023, 768), dtype=float32)

CPU times: total: 4h 8min 55s
Wall time: 32min 43s


In [7]:
%%time

# Загрузка эмбеддингов из файла
embeddings = np.load('embeddings.npy')
display(embeddings)

array([[ 3.63150924e-01,  4.89376448e-02, -2.64081329e-01, ...,
        -1.41593203e-01,  1.59179904e-02,  9.19513404e-05],
       [ 2.36416295e-01, -1.59501165e-01, -3.27798337e-01, ...,
        -2.89936006e-01,  1.19365111e-01, -1.62374461e-03],
       [ 3.75191450e-01, -1.13943860e-01, -2.40547046e-01, ...,
        -3.38919163e-01,  5.86938709e-02, -2.12656371e-02],
       ...,
       [ 3.40382367e-01,  6.64923042e-02, -1.63184434e-01, ...,
        -8.65626633e-02,  2.03403741e-01,  3.20906676e-02],
       [ 4.32092339e-01,  1.10913627e-02, -1.17306165e-01, ...,
         7.54014403e-02,  1.02739736e-01,  1.52742602e-02],
       [ 3.04277569e-01, -7.62157440e-02, -6.77585676e-02, ...,
        -5.43487109e-02,  2.44383678e-01, -1.41484402e-02]],
      shape=(7023, 768), dtype=float32)

CPU times: total: 172 ms
Wall time: 50.3 ms


In [8]:
%%time

### Пытаемся кластеризовать тексты
centered = embeddings - embeddings.mean()
pca = PCA(n_components=50)
pca_decomp = pca.fit_transform(centered)

n_clusters = 15
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(pca_decomp)
post_text['text_cluster'] = kmeans.labels_
dists_columns = [f'distance_to_cluster_{i}' for i in range(n_clusters)]
dists_df = pd.DataFrame(
    data=kmeans.transform(pca_decomp),
    columns=dists_columns
)

CPU times: total: 2.44 s
Wall time: 4.34 s


In [9]:
%%time

post_text = pd.concat((post_text, dists_df), axis=1)
post_text.drop(["text"], axis=1, inplace=True)
post_text = pd.get_dummies(post_text, columns=['topic'], drop_first=True)
display(post_text.info())
display(post_text)

<class 'pandas.DataFrame'>
RangeIndex: 7023 entries, 0 to 7022
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   post_id                 7023 non-null   int64  
 1   text_cluster            7023 non-null   int32  
 2   distance_to_cluster_0   7023 non-null   float32
 3   distance_to_cluster_1   7023 non-null   float32
 4   distance_to_cluster_2   7023 non-null   float32
 5   distance_to_cluster_3   7023 non-null   float32
 6   distance_to_cluster_4   7023 non-null   float32
 7   distance_to_cluster_5   7023 non-null   float32
 8   distance_to_cluster_6   7023 non-null   float32
 9   distance_to_cluster_7   7023 non-null   float32
 10  distance_to_cluster_8   7023 non-null   float32
 11  distance_to_cluster_9   7023 non-null   float32
 12  distance_to_cluster_10  7023 non-null   float32
 13  distance_to_cluster_11  7023 non-null   float32
 14  distance_to_cluster_12  7023 non-null   float32
 15

None

,post_id,text_cluster,distance_to_cluster_0,distance_to_cluster_1,distance_to_cluster_2,distance_to_cluster_3,distance_to_cluster_4,distance_to_cluster_5,distance_to_cluster_6,distance_to_cluster_7,...,distance_to_cluster_11,distance_to_cluster_12,distance_to_cluster_13,distance_to_cluster_14,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,9,3.275572,3.443513,3.469100,2.986742,1.948664,3.592508,3.383040,3.612148,...,3.400342,2.830196,2.372013,3.463925,False,False,False,False,False,False
1,2,9,3.192135,3.133693,3.244608,2.846341,2.196497,3.373540,3.331997,3.351419,...,3.336037,2.545622,2.334076,3.385151,False,False,False,False,False,False
2,3,9,2.974873,3.141050,3.392259,3.072776,1.817808,3.338186,3.459793,3.359396,...,3.375446,2.862795,2.404512,3.426456,False,False,False,False,False,False
3,4,4,3.202791,3.771742,4.060500,3.302979,2.426471,3.235161,3.748735,3.805814,...,3.738622,3.358398,2.818986,3.645685,False,False,False,False,False,False
4,5,4,2.978371,2.805885,3.242360,2.653685,1.472535,3.043557,2.781908,3.042039,...,2.817385,2.136082,2.047634,3.083163,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,11,3.082503,1.834552,3.396312,2.826148,3.000543,2.006451,1.805223,2.940562,...,1.308614,2.298392,2.735910,3.154152,False,False,True,False,False,False
7019,7316,11,2.887662,1.800025,3.383409,2.509571,2.963336,1.797055,1.512296,2.583630,...,0.929435,2.204309,2.440939,2.938148,False,False,True,False,False,False
7020,7317,11,2.693022,1.996829,3.464333,2.553049,3.183830,2.234073,2.017127,2.375831,...,1.542910,2.417808,2.809133,2.899021,False,False,True,False,False,False
7021,7318,6,3.359133,1.617927,3.410661,3.112874,3.194130,1.661280,1.049353,3.296159,...,1.469077,2.262688,2.982443,3.463953,False,False,True,False,False,False


CPU times: total: 250 ms
Wall time: 51.2 ms


## 🔧 4. Создание признаков

### Для пользователей:
- Нормализация количества пользователей по городам/странам
- One-Hot Encoding для возрастных групп и экспериментальных групп
- Кодирование категориальных переменных (OS, source)

### Для временных меток:
- Извлечение месяца и дня
- Нормализация временных признаков

In [10]:
%%time

# Количество пользователей в каждом городе и стране
user_data['city_count'] = user_data.groupby('city')['user_id'].transform('count')
user_data['country_count'] = user_data.groupby('country')['user_id'].transform('count')

# Кодируем 'os' и 'source' как 0 и 1
user_data['os'] = user_data['os'].map({'Android': 0, 'iOS': 1})
user_data['source'] = user_data['source'].map({'ads': 0, 'organic': 1})

# Разбиваем 'age' на категории и применяем OHE для 'age_group'
age_bins = [0, 18, 20, 22, 25, 30, 35, 40, float('inf')]
age_labels = ['<18', '18_20', '20_22', '22_25', '25_30', '30_35', '35_40', 'gt_40']
user_data['age_group'] = pd.cut(user_data['age'], bins=age_bins, labels=age_labels, right=False)
user_data = pd.get_dummies(user_data, columns=['age_group'], drop_first=True)

# Применяем OHE для 'exp_group'
user_data = pd.get_dummies(user_data, columns=['exp_group'], drop_first=True)
user_data.drop(['age', 'city', 'country'], axis=1, inplace=True)

# Преобразование bool в int64 для таблицы user_data
bool_columns_user = [
    'gender', 'os', 'source',
    'age_group_18_20', 'age_group_20_22', 'age_group_22_25',
    'age_group_25_30', 'age_group_30_35', 'age_group_35_40',
    'age_group_gt_40', 
    'exp_group_1', 'exp_group_2', 'exp_group_3', 'exp_group_4',
]

user_data[bool_columns_user] = user_data[bool_columns_user].astype('int64')

# Преобразование bool в int64 для таблицы post_text
bool_columns_post = [
    'topic_covid', 'topic_entertainment', 'topic_movie',
    'topic_politics', 'topic_sport', 'topic_tech'
]

post_text[bool_columns_post] = post_text[bool_columns_post].astype('int64')

feed_data['timestamp'] = pd.to_datetime(feed_data['timestamp'])
feed_data.sort_values(by='timestamp', ascending=True, inplace=True)
feed_data['month'] = feed_data['timestamp'].dt.month / 12
feed_data['day'] = feed_data['timestamp'].dt.day / 31
feed_data.drop(['timestamp', 'action'], axis=1, inplace=True)
feed_data['target'] = feed_data['target'].astype(int)

user_data[['city_count', 'country_count']] = StandardScaler().fit_transform(user_data[['city_count', 'country_count']])
user_data = user_data.rename(columns={"city_count": "city_countscaled",
                                      "country_count": "country_countscaled"})

CPU times: total: 11.4 s
Wall time: 11.5 s


In [11]:
%%time

display(user_data.info(),
        post_text.info(),
        feed_data.info())

display(user_data,
        post_text,
        feed_data)

<class 'pandas.DataFrame'>
RangeIndex: 163205 entries, 0 to 163204
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              163205 non-null  int64  
 1   gender               163205 non-null  int64  
 2   os                   163205 non-null  int64  
 3   source               163205 non-null  int64  
 4   city_countscaled     163205 non-null  float64
 5   country_countscaled  163205 non-null  float64
 6   age_group_18_20      163205 non-null  int64  
 7   age_group_20_22      163205 non-null  int64  
 8   age_group_22_25      163205 non-null  int64  
 9   age_group_25_30      163205 non-null  int64  
 10  age_group_30_35      163205 non-null  int64  
 11  age_group_35_40      163205 non-null  int64  
 12  age_group_gt_40      163205 non-null  int64  
 13  exp_group_1          163205 non-null  int64  
 14  exp_group_2          163205 non-null  int64  
 15  exp_group_3          163205 

None

None

None

,user_id,gender,os,source,city_countscaled,country_countscaled,age_group_18_20,age_group_20_22,age_group_22_25,age_group_25_30,age_group_30_35,age_group_35_40,age_group_gt_40,exp_group_1,exp_group_2,exp_group_3,exp_group_4
0,200,1,0,0,-0.492196,0.375419,0,0,0,0,1,0,0,0,0,1,0
1,201,0,0,0,-0.461974,0.375419,0,0,0,0,0,1,0,0,0,0,0
2,202,1,0,0,-0.435580,0.375419,0,0,0,0,0,0,0,0,0,0,1
3,203,0,1,0,2.496407,0.375419,1,0,0,0,0,0,0,1,0,0,0
4,204,0,0,0,-0.480572,0.375419,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,0,1,-0.417118,0.375419,0,0,0,0,0,1,0,0,0,0,1
163201,168549,0,0,1,-0.409050,0.375419,1,0,0,0,0,0,0,0,1,0,0
163202,168550,1,0,1,-0.231818,0.375419,0,0,0,0,0,0,1,0,0,0,1
163203,168551,0,1,1,2.496407,0.375419,0,0,0,0,0,1,0,0,0,1,0


,post_id,text_cluster,distance_to_cluster_0,distance_to_cluster_1,distance_to_cluster_2,distance_to_cluster_3,distance_to_cluster_4,distance_to_cluster_5,distance_to_cluster_6,distance_to_cluster_7,...,distance_to_cluster_11,distance_to_cluster_12,distance_to_cluster_13,distance_to_cluster_14,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,9,3.275572,3.443513,3.469100,2.986742,1.948664,3.592508,3.383040,3.612148,...,3.400342,2.830196,2.372013,3.463925,0,0,0,0,0,0
1,2,9,3.192135,3.133693,3.244608,2.846341,2.196497,3.373540,3.331997,3.351419,...,3.336037,2.545622,2.334076,3.385151,0,0,0,0,0,0
2,3,9,2.974873,3.141050,3.392259,3.072776,1.817808,3.338186,3.459793,3.359396,...,3.375446,2.862795,2.404512,3.426456,0,0,0,0,0,0
3,4,4,3.202791,3.771742,4.060500,3.302979,2.426471,3.235161,3.748735,3.805814,...,3.738622,3.358398,2.818986,3.645685,0,0,0,0,0,0
4,5,4,2.978371,2.805885,3.242360,2.653685,1.472535,3.043557,2.781908,3.042039,...,2.817385,2.136082,2.047634,3.083163,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,11,3.082503,1.834552,3.396312,2.826148,3.000543,2.006451,1.805223,2.940562,...,1.308614,2.298392,2.735910,3.154152,0,0,1,0,0,0
7019,7316,11,2.887662,1.800025,3.383409,2.509571,2.963336,1.797055,1.512296,2.583630,...,0.929435,2.204309,2.440939,2.938148,0,0,1,0,0,0
7020,7317,11,2.693022,1.996829,3.464333,2.553049,3.183830,2.234073,2.017127,2.375831,...,1.542910,2.417808,2.809133,2.899021,0,0,1,0,0,0
7021,7318,6,3.359133,1.617927,3.410661,3.112874,3.194130,1.661280,1.049353,3.296159,...,1.469077,2.262688,2.982443,3.463953,0,0,1,0,0,0


,user_id,post_id,target,month,day
2304354,66609,3270,1,0.833333,0.032258
1999320,15471,2810,0,0.833333,0.032258
784528,80305,961,0,0.833333,0.032258
2304355,66609,3270,0,0.833333,0.032258
2304356,66609,4320,0,0.833333,0.032258
...,...,...,...,...,...
6426675,45075,3027,0,1.000000,0.935484
3019649,116395,5771,0,1.000000,0.935484
4359850,53559,587,0,1.000000,0.935484
7435524,109595,1702,0,1.000000,0.935484


CPU times: total: 78.1 ms
Wall time: 72.7 ms


In [12]:
%%time

user_data.to_csv(PROCESSED_DATA_DIR / "processed_user_data.csv", index=False)
post_text.to_csv(PROCESSED_DATA_DIR / "processed_post_text_dl.csv", index=False)

CPU times: total: 1.48 s
Wall time: 1.72 s


## 🤖 5. Обучение CatBoost модели

### Параметры модели:
- Алгоритм: CatBoostClassifier
- Итерации: 500
- Learning rate: 0.07
- Глубина: 6
- Cat features: 20 категориальных признаков
- Метрика: AUC
- Балансировка классов: включена

In [13]:
%%time

user_post_feed = feed_data.merge(user_data, on="user_id", how="left")
user_post_feed = user_post_feed.merge(post_text, on="post_id", how="left")

CPU times: total: 5.59 s
Wall time: 6.26 s


In [14]:
%%time

display(user_post_feed.info())
display(user_post_feed)

<class 'pandas.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 43 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   user_id                 int64  
 1   post_id                 int64  
 2   target                  int64  
 3   month                   float64
 4   day                     float64
 5   gender                  int64  
 6   os                      int64  
 7   source                  int64  
 8   city_countscaled        float64
 9   country_countscaled     float64
 10  age_group_18_20         int64  
 11  age_group_20_22         int64  
 12  age_group_22_25         int64  
 13  age_group_25_30         int64  
 14  age_group_30_35         int64  
 15  age_group_35_40         int64  
 16  age_group_gt_40         int64  
 17  exp_group_1             int64  
 18  exp_group_2             int64  
 19  exp_group_3             int64  
 20  exp_group_4             int64  
 21  text_cluster            int32  
 22  dist

None

,user_id,post_id,target,month,day,gender,os,source,city_countscaled,country_countscaled,...,distance_to_cluster_11,distance_to_cluster_12,distance_to_cluster_13,distance_to_cluster_14,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,66609,3270,1,0.833333,0.032258,1,0,0,-0.485358,0.375419,...,2.865479,2.992789,2.837126,2.394862,1,0,0,0,0,0
1,15471,2810,0,0.833333,0.032258,1,1,0,-0.444743,0.375419,...,2.599644,2.617199,2.583309,2.257900,1,0,0,0,0,0
2,80305,961,0,0.833333,0.032258,0,1,0,-0.297596,0.375419,...,2.279086,2.608468,2.389139,3.322142,0,0,0,1,0,0
3,66609,3270,0,0.833333,0.032258,1,0,0,-0.485358,0.375419,...,2.865479,2.992789,2.837126,2.394862,1,0,0,0,0,0
4,66609,4320,0,0.833333,0.032258,1,0,0,-0.485358,0.375419,...,1.785501,2.075130,2.540845,2.966598,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,45075,3027,0,1.000000,0.935484,1,1,0,-0.470315,-2.695570,...,2.602210,2.910203,2.740366,2.206932,1,0,0,0,0,0
9999996,116395,5771,0,1.000000,0.935484,1,0,1,-0.294451,0.375419,...,1.319838,2.392977,2.507815,3.223293,0,0,1,0,0,0
9999997,53559,587,0,1.000000,0.935484,0,0,0,-0.468401,-2.732798,...,2.895838,2.054137,2.665680,3.268947,0,1,0,0,0,0
9999998,109595,1702,0,1.000000,0.935484,1,0,1,-0.314007,0.375419,...,3.216987,2.715987,3.126413,3.884958,0,0,0,0,1,0


CPU times: total: 31.2 ms
Wall time: 35.1 ms


In [15]:
%%time

X = user_post_feed.drop(['target'], axis=1)
y = user_post_feed['target']

cat_features = ['gender',
                'os',
                'source',
                'age_group_18_20',
                'age_group_20_22',
                'age_group_22_25',
                'age_group_25_30',
                'age_group_30_35',
                'age_group_35_40',
                'age_group_gt_40',
                'exp_group_1',
                'exp_group_2',
                'exp_group_3',
                'exp_group_4',
                'text_cluster',
                'topic_covid',
                'topic_entertainment',
                'topic_movie',
                'topic_politics',
                'topic_sport',
                'topic_tech']

CPU times: total: 0 ns
Wall time: 958 μs


In [16]:
%%time

name = "CatBoostClassifier"

model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.07,
    depth=6,
    cat_features=cat_features,
    eval_metric='AUC',
    auto_class_weights='Balanced',
    random_seed=42
)

CPU times: total: 62.5 ms
Wall time: 78 ms


In [17]:
%%time

X_train, y_train = X.iloc[:8500000], y.iloc[:8500000]
X_test, y_test = X.iloc[8500000:], y.iloc[8500000:]

CPU times: total: 0 ns
Wall time: 957 μs


In [18]:
%%time

start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()

# Время обучения
print(round(end_time - start_time, 2))

0:	total: 4.25s	remaining: 35m 21s
1:	total: 8.65s	remaining: 35m 53s
2:	total: 11.6s	remaining: 31m 56s
3:	total: 14.2s	remaining: 29m 21s
4:	total: 16.3s	remaining: 26m 56s
5:	total: 20.8s	remaining: 28m 35s
6:	total: 23.8s	remaining: 27m 53s
7:	total: 25.9s	remaining: 26m 32s
8:	total: 28s	remaining: 25m 29s
9:	total: 30.7s	remaining: 25m 4s
10:	total: 33.6s	remaining: 24m 52s
11:	total: 35.8s	remaining: 24m 15s
12:	total: 38.4s	remaining: 23m 57s
13:	total: 41.2s	remaining: 23m 50s
14:	total: 44.8s	remaining: 24m 7s
15:	total: 48.3s	remaining: 24m 20s
16:	total: 52.3s	remaining: 24m 46s
17:	total: 55.4s	remaining: 24m 42s
18:	total: 59.4s	remaining: 25m 4s
19:	total: 1m 3s	remaining: 25m 18s
20:	total: 1m 6s	remaining: 25m 17s
21:	total: 1m 10s	remaining: 25m 35s
22:	total: 1m 14s	remaining: 25m 37s
23:	total: 1m 18s	remaining: 25m 54s
24:	total: 1m 22s	remaining: 26m 4s
25:	total: 1m 26s	remaining: 26m 19s
26:	total: 1m 30s	remaining: 26m 29s
27:	total: 1m 34s	remaining: 26m 34s
2

In [19]:
%%time

# Сохранение модели
model.save_model("catboost_dl_model.cbm")

CPU times: total: 0 ns
Wall time: 3.75 ms


## 📊 6. Оценка качества модели

### Метрики:
- AUC-ROC
- Precision-Recall AUC  
- F1-score
- Accuracy, Precision, Recall

### Важность признаков:
- Анализ наиболее влиятельных признаков на предсказание

In [20]:
import os

from catboost import CatBoostClassifier


def get_model_path(path: str) -> str:
    if os.environ.get("IS_LMS") == "1":
        MODEL_PATH = "/workdir/user_input/model"
    else:
        MODEL_PATH = path
    return MODEL_PATH


def load_models():
    model_path = get_model_path("catboost_dl_model.cbm")
    model = CatBoostClassifier()
    model.load_model(model_path)
    return model

In [21]:
%%time

# Загрузка модели
loaded_model = load_models()

CPU times: total: 0 ns
Wall time: 56 ms


In [22]:
%%time

# Таблица для хранения коэффициентов
feature_importance_df = pd.DataFrame(index=X.columns)

# Таблица для хранения метрик
metrics_df = pd.DataFrame(columns=["learn_time",
                                   "train_model_score", "test_model_score",
                                   "train_auc_roc", "test_auc_roc", "train_pr_auc", "test_pr_auc",
                                   "train_f1", "test_f1", "train_accuracy", "test_accuracy",
                                   "train_precision", "test_precision", "train_recall", "test_recall"])

CPU times: total: 15.6 ms
Wall time: 7.21 ms


In [23]:
%%time

print('Оцениваем модель на трейне')

t0 = time.time()
train_model_score = loaded_model.score(X_train, y_train)
t1 = time.time()
print('\t1\t', round(t1 - t0, 2))

t2 = time.time()
y_train_pred = loaded_model.predict(X_train)
t3 = time.time()
print('\t2\t', round(t3 - t2, 2))

t4 = time.time()
y_train_proba = loaded_model.predict_proba(X_train)[:, 1] if hasattr(loaded_model, 'predict_proba') else y_train_pred
t5 = time.time()
print('\t3\t', round(t5 - t4, 2))

t6 = time.time()
train_auc_roc = roc_auc_score(y_train, y_train_proba)
t7 = time.time()
print('\t4\t', round(t7 - t6, 2))

t8 = time.time()
train_pr_auc = average_precision_score(y_train, y_train_proba)
t9 = time.time()
print('\t5\t', round(t9 - t8, 2))

t10 = time.time()
train_f1 = f1_score(y_train, y_train_pred)
t11 = time.time()
print('\t6\t', round(t11 - t10, 2))

t12 = time.time()
train_accuracy = accuracy_score(y_train, y_train_pred)
t13 = time.time()
print('\t7\t', round(t13 - t12, 2))

t14 = time.time()
train_precision = precision_score(y_train, y_train_pred)
t15 = time.time()
print('\t8\t', round(t15 - t14, 2))

t16 = time.time()
train_recall = recall_score(y_train, y_train_pred)
t17 = time.time()
print('\t9\t', round(t17 - t16, 2))

print('Оценили модель на трейне')
print('Оцениваем модель на тесте')

t18 = time.time()
test_model_score = loaded_model.score(X_test, y_test)
t19 = time.time()
print('\t10\t', round(t19 - t18, 2))

t20 = time.time()
y_test_pred = loaded_model.predict(X_test)
t21 = time.time()
print('\t11\t', round(t21 - t20, 2))

t22 = time.time()
y_test_proba = loaded_model.predict_proba(X_test)[:, 1] if hasattr(loaded_model, 'predict_proba') else y_test_pred
t23 = time.time()
print('\t12\t', round(t23 - t22, 2))

t24 = time.time()
test_auc_roc = roc_auc_score(y_test, y_test_proba)
t25 = time.time()
print('\t13\t', round(t25 - t24, 2))

t26 = time.time()
test_pr_auc = average_precision_score(y_test, y_test_proba)
t27 = time.time()
print('\t14\t', round(t27 - t26, 2))

t28 = time.time()
test_f1 = f1_score(y_test, y_test_pred)
t29 = time.time()
print('\t15\t', round(t29 - t28, 2))

t30 = time.time()
test_accuracy = accuracy_score(y_test, y_test_pred)
t31 = time.time()
print('\t16\t', round(t31 - t30, 2))

t32 = time.time()
test_precision = precision_score(y_test, y_test_pred)
t33 = time.time()
print('\t17\t', round(t33 - t32, 2))

t34 = time.time()
test_recall = recall_score(y_test, y_test_pred)
t35 = time.time()
print('\t18\t', round(t35 - t34, 2))

print('Оценили модель на тесте')
print('Запиcываем метрики в таблицу')

# Формируем DataFrame с нужной структурой
metrics = {
    "model_score": [train_model_score, test_model_score],
    "auc_roc": [train_auc_roc, test_auc_roc],
    "pr_auc": [train_pr_auc, test_pr_auc],
    "f1": [train_f1, test_f1],
    "accuracy": [train_accuracy, test_accuracy],
    "precision": [train_precision, test_precision],
    "recall": [train_recall, test_recall]
}

metrics_df = pd.DataFrame(metrics, index=["train", "test"])

print('Запиcали метрики в таблицу')
print('Запиcываем коэффициенты и важности признаков в таблицу')

importances = loaded_model.get_feature_importance()

# Создаем DataFrame с важностью признаков
feature_importance_df = pd.DataFrame({
    'feature': X_train.columns.tolist() if hasattr(X_train, 'columns') else [f'feature_{i}' 
                                                                             for i in range(len(importances))],
    'importance': importances
})

print('Записали коэффициенты и важности признаков в таблицу')

Оцениваем модель на трейне
	1	 134.92
	2	 129.65
	3	 123.5
	4	 5.86
	5	 4.96
	6	 1.67
	7	 0.38
	8	 1.64
	9	 1.63
Оценили модель на трейне
Оцениваем модель на тесте
	10	 22.45
	11	 23.26
	12	 22.14
	13	 0.9
	14	 0.73
	15	 0.3
	16	 0.07
	17	 0.29
	18	 0.31
Оценили модель на тесте
Запиcываем метрики в таблицу
Запиcали метрики в таблицу
Запиcываем коэффициенты и важности признаков в таблицу
Записали коэффициенты и важности признаков в таблицу
CPU times: total: 9min 52s
Wall time: 7min 54s


In [24]:
%%time

# Вывод результатов
print("Feature Importance Table:")
display(feature_importance_df.sort_values(by='importance', ascending=False))

print("\nModel Metrics Table:")
display(metrics_df)

Feature Importance Table:


,feature,importance
7,city_countscaled,19.602521
8,country_countscaled,7.373620
15,age_group_gt_40,7.129731
13,age_group_30_35,6.401545
2,month,5.844604
16,exp_group_1,5.304380
17,exp_group_2,5.039003
14,age_group_35_40,4.735635
32,distance_to_cluster_11,4.032566
1,post_id,3.574938



Model Metrics Table:


,model_score,auc_roc,pr_auc,f1,accuracy,precision,recall
train,0.587461,0.649008,0.161080,0.241410,0.587461,0.149283,0.630525
test,0.486502,0.624952,0.172671,0.254436,0.486502,0.154382,0.723021


CPU times: total: 31.2 ms
Wall time: 29.2 ms


## 💾 7. Экспорт обработанных данных в PostgreSQL

### Создаваемые таблицы:
1. `processed_user_data` - обработанные признаки пользователей
2. `processed_post_text_dl` - улучшенная NLP обработка
3. `processed_feed_data_like` - данные о лайках
4. `clear_post_text` - исходные тексты постов

### Импорт библиотек

In [1]:
import os
import time
import warnings
from io import StringIO
from pathlib import Path

import pandas as pd
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy import text

warnings.filterwarnings('ignore')

### Определяем пути

In [2]:
%%time

# Определяем пути
PROJECT_ROOT = Path().resolve().parent
RAW_DATA_DIR = PROJECT_ROOT / "part3-nlp-enhanced" / "raw_data"
PROCESSED_DATA_DIR = PROJECT_ROOT / "part3-nlp-enhanced" / "processed_data"

CPU times: total: 0 ns
Wall time: 747 μs


### Загружаем обработанные и необработанные данные

In [3]:
%%time

processed_user_data = pd.read_csv(PROCESSED_DATA_DIR / "processed_user_data.csv")
processed_post_text_dl = pd.read_csv(PROCESSED_DATA_DIR / "processed_post_text_dl.csv")
feed_data_like = pd.read_csv(RAW_DATA_DIR / "feed_data_like.csv")
clear_post_text = pd.read_csv(RAW_DATA_DIR / "post_text.csv")

CPU times: total: 9.62 s
Wall time: 10 s


In [4]:
%%time

display(processed_user_data.info(),
        processed_post_text_dl.info(),
        feed_data_like.info(),
        clear_post_text.info())

display(processed_user_data,
        processed_post_text_dl,
        feed_data_like,
        clear_post_text)

<class 'pandas.DataFrame'>
RangeIndex: 163205 entries, 0 to 163204
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              163205 non-null  int64  
 1   gender               163205 non-null  int64  
 2   os                   163205 non-null  int64  
 3   source               163205 non-null  int64  
 4   city_countscaled     163205 non-null  float64
 5   country_countscaled  163205 non-null  float64
 6   age_group_18_20      163205 non-null  int64  
 7   age_group_20_22      163205 non-null  int64  
 8   age_group_22_25      163205 non-null  int64  
 9   age_group_25_30      163205 non-null  int64  
 10  age_group_30_35      163205 non-null  int64  
 11  age_group_35_40      163205 non-null  int64  
 12  age_group_gt_40      163205 non-null  int64  
 13  exp_group_1          163205 non-null  int64  
 14  exp_group_2          163205 non-null  int64  
 15  exp_group_3          163205 

None

None

None

None

,user_id,gender,os,source,city_countscaled,country_countscaled,age_group_18_20,age_group_20_22,age_group_22_25,age_group_25_30,age_group_30_35,age_group_35_40,age_group_gt_40,exp_group_1,exp_group_2,exp_group_3,exp_group_4
0,200,1,0,0,-0.492196,0.375419,0,0,0,0,1,0,0,0,0,1,0
1,201,0,0,0,-0.461974,0.375419,0,0,0,0,0,1,0,0,0,0,0
2,202,1,0,0,-0.435580,0.375419,0,0,0,0,0,0,0,0,0,0,1
3,203,0,1,0,2.496407,0.375419,1,0,0,0,0,0,0,1,0,0,0
4,204,0,0,0,-0.480572,0.375419,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,0,1,-0.417118,0.375419,0,0,0,0,0,1,0,0,0,0,1
163201,168549,0,0,1,-0.409050,0.375419,1,0,0,0,0,0,0,0,1,0,0
163202,168550,1,0,1,-0.231818,0.375419,0,0,0,0,0,0,1,0,0,0,1
163203,168551,0,1,1,2.496407,0.375419,0,0,0,0,0,1,0,0,0,1,0


,post_id,text_cluster,distance_to_cluster_0,distance_to_cluster_1,distance_to_cluster_2,distance_to_cluster_3,distance_to_cluster_4,distance_to_cluster_5,distance_to_cluster_6,distance_to_cluster_7,...,distance_to_cluster_11,distance_to_cluster_12,distance_to_cluster_13,distance_to_cluster_14,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,9,3.275572,3.443513,3.469100,2.986742,1.948664,3.592508,3.383040,3.612148,...,3.400342,2.830196,2.372013,3.463925,0,0,0,0,0,0
1,2,9,3.192135,3.133694,3.244608,2.846341,2.196497,3.373540,3.331997,3.351419,...,3.336037,2.545622,2.334076,3.385151,0,0,0,0,0,0
2,3,9,2.974873,3.141050,3.392259,3.072776,1.817808,3.338185,3.459793,3.359396,...,3.375446,2.862795,2.404512,3.426456,0,0,0,0,0,0
3,4,4,3.202792,3.771742,4.060500,3.302980,2.426471,3.235161,3.748735,3.805814,...,3.738622,3.358398,2.818986,3.645685,0,0,0,0,0,0
4,5,4,2.978371,2.805885,3.242360,2.653685,1.472535,3.043557,2.781908,3.042039,...,2.817385,2.136082,2.047634,3.083162,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,11,3.082503,1.834552,3.396313,2.826148,3.000543,2.006451,1.805223,2.940562,...,1.308614,2.298392,2.735910,3.154152,0,0,1,0,0,0
7019,7316,11,2.887662,1.800025,3.383409,2.509571,2.963336,1.797055,1.512296,2.583630,...,0.929435,2.204309,2.440939,2.938149,0,0,1,0,0,0
7020,7317,11,2.693023,1.996829,3.464333,2.553049,3.183830,2.234073,2.017127,2.375831,...,1.542911,2.417808,2.809133,2.899021,0,0,1,0,0,0
7021,7318,6,3.359132,1.617927,3.410661,3.112874,3.194130,1.661280,1.049354,3.296159,...,1.469077,2.262688,2.982443,3.463953,0,0,1,0,0,0


,timestamp,user_id,post_id,action,target
0,2021-12-19 21:58:49,72194,2323,like,0
1,2021-11-20 18:36:14,162059,4190,like,0
2,2021-11-20 18:46:11,162059,2033,like,0
3,2021-11-20 18:52:07,162059,1581,like,0
4,2021-11-20 18:59:02,162059,1172,like,0
...,...,...,...,...,...
8206340,2021-12-19 21:44:12,72194,404,like,0
8206341,2021-12-19 21:50:35,72194,5028,like,0
8206342,2021-12-19 21:51:05,72194,3876,like,0
8206343,2021-12-19 21:53:17,72194,7129,like,0


,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


CPU times: total: 156 ms
Wall time: 143 ms


In [5]:
%%time

feed_data_like['timestamp'] = pd.to_datetime(feed_data_like['timestamp'])
feed_data_like.sort_values(by='timestamp', ascending=True, inplace=True)
feed_data_like.drop(['timestamp', 'action', 'target'], axis=1, inplace=True)

CPU times: total: 7.8 s
Wall time: 8.05 s


In [6]:
%%time

feed_data_like.to_csv(PROCESSED_DATA_DIR / "processed_feed_data_like.csv", index=False)

CPU times: total: 7.61 s
Wall time: 7.83 s


In [7]:
%%time

processed_feed_data_like = pd.read_csv(PROCESSED_DATA_DIR / "processed_feed_data_like.csv")

CPU times: total: 3.05 s
Wall time: 3.25 s


In [8]:
%%time

display(processed_user_data.info(),
        processed_post_text_dl.info(),
        processed_feed_data_like.info(),
        clear_post_text.info())

display(processed_user_data,
        processed_post_text_dl,
        processed_feed_data_like,
        clear_post_text)

<class 'pandas.DataFrame'>
RangeIndex: 163205 entries, 0 to 163204
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              163205 non-null  int64  
 1   gender               163205 non-null  int64  
 2   os                   163205 non-null  int64  
 3   source               163205 non-null  int64  
 4   city_countscaled     163205 non-null  float64
 5   country_countscaled  163205 non-null  float64
 6   age_group_18_20      163205 non-null  int64  
 7   age_group_20_22      163205 non-null  int64  
 8   age_group_22_25      163205 non-null  int64  
 9   age_group_25_30      163205 non-null  int64  
 10  age_group_30_35      163205 non-null  int64  
 11  age_group_35_40      163205 non-null  int64  
 12  age_group_gt_40      163205 non-null  int64  
 13  exp_group_1          163205 non-null  int64  
 14  exp_group_2          163205 non-null  int64  
 15  exp_group_3          163205 

None

None

None

None

,user_id,gender,os,source,city_countscaled,country_countscaled,age_group_18_20,age_group_20_22,age_group_22_25,age_group_25_30,age_group_30_35,age_group_35_40,age_group_gt_40,exp_group_1,exp_group_2,exp_group_3,exp_group_4
0,200,1,0,0,-0.492196,0.375419,0,0,0,0,1,0,0,0,0,1,0
1,201,0,0,0,-0.461974,0.375419,0,0,0,0,0,1,0,0,0,0,0
2,202,1,0,0,-0.435580,0.375419,0,0,0,0,0,0,0,0,0,0,1
3,203,0,1,0,2.496407,0.375419,1,0,0,0,0,0,0,1,0,0,0
4,204,0,0,0,-0.480572,0.375419,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,0,1,-0.417118,0.375419,0,0,0,0,0,1,0,0,0,0,1
163201,168549,0,0,1,-0.409050,0.375419,1,0,0,0,0,0,0,0,1,0,0
163202,168550,1,0,1,-0.231818,0.375419,0,0,0,0,0,0,1,0,0,0,1
163203,168551,0,1,1,2.496407,0.375419,0,0,0,0,0,1,0,0,0,1,0


,post_id,text_cluster,distance_to_cluster_0,distance_to_cluster_1,distance_to_cluster_2,distance_to_cluster_3,distance_to_cluster_4,distance_to_cluster_5,distance_to_cluster_6,distance_to_cluster_7,...,distance_to_cluster_11,distance_to_cluster_12,distance_to_cluster_13,distance_to_cluster_14,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,9,3.275572,3.443513,3.469100,2.986742,1.948664,3.592508,3.383040,3.612148,...,3.400342,2.830196,2.372013,3.463925,0,0,0,0,0,0
1,2,9,3.192135,3.133694,3.244608,2.846341,2.196497,3.373540,3.331997,3.351419,...,3.336037,2.545622,2.334076,3.385151,0,0,0,0,0,0
2,3,9,2.974873,3.141050,3.392259,3.072776,1.817808,3.338185,3.459793,3.359396,...,3.375446,2.862795,2.404512,3.426456,0,0,0,0,0,0
3,4,4,3.202792,3.771742,4.060500,3.302980,2.426471,3.235161,3.748735,3.805814,...,3.738622,3.358398,2.818986,3.645685,0,0,0,0,0,0
4,5,4,2.978371,2.805885,3.242360,2.653685,1.472535,3.043557,2.781908,3.042039,...,2.817385,2.136082,2.047634,3.083162,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,11,3.082503,1.834552,3.396313,2.826148,3.000543,2.006451,1.805223,2.940562,...,1.308614,2.298392,2.735910,3.154152,0,0,1,0,0,0
7019,7316,11,2.887662,1.800025,3.383409,2.509571,2.963336,1.797055,1.512296,2.583630,...,0.929435,2.204309,2.440939,2.938149,0,0,1,0,0,0
7020,7317,11,2.693023,1.996829,3.464333,2.553049,3.183830,2.234073,2.017127,2.375831,...,1.542911,2.417808,2.809133,2.899021,0,0,1,0,0,0
7021,7318,6,3.359132,1.617927,3.410661,3.112874,3.194130,1.661280,1.049354,3.296159,...,1.469077,2.262688,2.982443,3.463953,0,0,1,0,0,0


,user_id,post_id
0,156604,573
1,142997,6445
2,122594,4554
3,8663,3837
4,94081,5671
...,...,...
8206340,113194,1629
8206341,84572,6836
8206342,140395,4485
8206343,33284,2093


,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


CPU times: total: 109 ms
Wall time: 107 ms


In [9]:
%%time

load_dotenv()
SQLALCHEMY_DATABASE_URL = os.getenv("SQLALCHEMY_DATABASE_URL")

engine = create_engine(SQLALCHEMY_DATABASE_URL)

CPU times: total: 78.1 ms
Wall time: 76.6 ms


In [10]:
%%time

with engine.begin() as conn:
    conn.execute(text("""
        DROP TABLE IF EXISTS processed_user_data;
        DROP TABLE IF EXISTS processed_post_text_dl;
        DROP TABLE IF EXISTS processed_feed_data_like;
        DROP TABLE IF EXISTS clear_post_text;
    """))

CPU times: total: 0 ns
Wall time: 4.08 s


In [11]:
%%time

with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE processed_user_data (
            user_id BIGINT,
            gender INT,
            os INT,
            source INT,
            city_countscaled FLOAT,
            country_countscaled FLOAT,
            age_group_18_20 INT,
            age_group_20_22 INT,
            age_group_22_25 INT,
            age_group_25_30 INT,
            age_group_30_35 INT,
            age_group_35_40 INT,
            age_group_gt_40 INT,
            exp_group_1 INT,
            exp_group_2 INT,
            exp_group_3 INT,
            exp_group_4 INT
        );
    """))

CPU times: total: 15.6 ms
Wall time: 570 ms


In [12]:
%%time

with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE processed_post_text_dl (
            post_id BIGINT,
            text_cluster INT,
            distance_to_cluster_0 FLOAT,
            distance_to_cluster_1 FLOAT,
            distance_to_cluster_2 FLOAT,
            distance_to_cluster_3 FLOAT,
            distance_to_cluster_4 FLOAT,
            distance_to_cluster_5 FLOAT,
            distance_to_cluster_6 FLOAT,
            distance_to_cluster_7 FLOAT,
            distance_to_cluster_8 FLOAT,
            distance_to_cluster_9 FLOAT,
            distance_to_cluster_10 FLOAT,
            distance_to_cluster_11 FLOAT,
            distance_to_cluster_12 FLOAT,
            distance_to_cluster_13 FLOAT,
            distance_to_cluster_14 FLOAT,
            topic_covid INT,
            topic_entertainment INT,
            topic_movie INT,
            topic_politics INT,
            topic_sport INT,
            topic_tech INT
        );
    """))

CPU times: total: 0 ns
Wall time: 570 ms


In [13]:
%%time

with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE processed_feed_data_like (
            user_id BIGINT,
            post_id BIGINT
        );
    """))

CPU times: total: 0 ns
Wall time: 611 ms


In [14]:
%%time

with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE clear_post_text (
            post_id BIGINT,
            text TEXT,
            topic TEXT
        );
    """))

CPU times: total: 0 ns
Wall time: 589 ms


In [15]:
%%time

def copy_dataframe(df, table_name, conn_str):
    conn = psycopg2.connect(conn_str)
    cur = conn.cursor()

    buffer = StringIO()
    df.to_csv(buffer, index=False, header=False)
    buffer.seek(0)

    cur.copy_expert(
        f"COPY {table_name} FROM STDIN WITH CSV",
        buffer
    )

    conn.commit()
    cur.close()
    conn.close()

CPU times: total: 0 ns
Wall time: 5.01 μs


In [16]:
t0 = time.time()
copy_dataframe(processed_user_data, "processed_user_data", SQLALCHEMY_DATABASE_URL)
t1 = time.time()
print(round(t1 - t0, 2))

4.62


In [17]:
t2 = time.time()
copy_dataframe(processed_post_text_dl, "processed_post_text_dl", SQLALCHEMY_DATABASE_URL)
t3 = time.time()
print(round(t3 - t2, 2))

2.82


In [18]:
t4 = time.time()
copy_dataframe(processed_feed_data_like, "processed_feed_data_like", SQLALCHEMY_DATABASE_URL)
t5 = time.time()
print(round(t5 - t4, 2))

212.93


In [19]:
t6 = time.time()
copy_dataframe(clear_post_text, "clear_post_text", SQLALCHEMY_DATABASE_URL)
t7 = time.time()
print(round(t7 - t6, 2))

18.24


In [20]:
%%time

def copy_query_to_df(query, conn_str):
    conn = psycopg2.connect(conn_str)
    cur = conn.cursor()

    buffer = StringIO()
    cur.copy_expert(f"COPY ({query}) TO STDOUT WITH CSV HEADER", buffer)
    buffer.seek(0)

    df = pd.read_csv(buffer)

    cur.close()
    conn.close()
    return df

CPU times: total: 0 ns
Wall time: 20.3 μs


In [21]:
t0 = time.time()
processed_user_data_from_db = copy_query_to_df(
    "SELECT * FROM processed_user_data",
    SQLALCHEMY_DATABASE_URL
)
t1 = time.time()
print(round(t1 - t0, 2))

5.65


In [22]:
t2 = time.time()
processed_post_text_dl_from_db = copy_query_to_df(
    "SELECT * FROM processed_post_text_dl",
    SQLALCHEMY_DATABASE_URL
)
t3 = time.time()
print(round(t3 - t2, 2))

2.55


In [23]:
t4 = time.time()
processed_feed_data_like_from_db = copy_query_to_df(
    "SELECT * FROM processed_feed_data_like LIMIT 100000",
    SQLALCHEMY_DATABASE_URL
)
t5 = time.time()
print(round(t5 - t4, 2))

4.54


In [24]:
t6 = time.time()
clear_post_text_from_db = copy_query_to_df(
    "SELECT * FROM clear_post_text",
    SQLALCHEMY_DATABASE_URL
)
t7 = time.time()
print(round(t7 - t6, 2))

4.44


In [25]:
display(processed_user_data_from_db.info(),
        processed_post_text_dl_from_db.info(),
        processed_feed_data_like_from_db.info(),
        clear_post_text_from_db.info())

display(processed_user_data_from_db,
        processed_post_text_dl_from_db,
        processed_feed_data_like_from_db,
        clear_post_text_from_db)

<class 'pandas.DataFrame'>
RangeIndex: 163205 entries, 0 to 163204
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              163205 non-null  int64  
 1   gender               163205 non-null  int64  
 2   os                   163205 non-null  int64  
 3   source               163205 non-null  int64  
 4   city_countscaled     163205 non-null  float64
 5   country_countscaled  163205 non-null  float64
 6   age_group_18_20      163205 non-null  int64  
 7   age_group_20_22      163205 non-null  int64  
 8   age_group_22_25      163205 non-null  int64  
 9   age_group_25_30      163205 non-null  int64  
 10  age_group_30_35      163205 non-null  int64  
 11  age_group_35_40      163205 non-null  int64  
 12  age_group_gt_40      163205 non-null  int64  
 13  exp_group_1          163205 non-null  int64  
 14  exp_group_2          163205 non-null  int64  
 15  exp_group_3          163205 

None

None

None

None

,user_id,gender,os,source,city_countscaled,country_countscaled,age_group_18_20,age_group_20_22,age_group_22_25,age_group_25_30,age_group_30_35,age_group_35_40,age_group_gt_40,exp_group_1,exp_group_2,exp_group_3,exp_group_4
0,200,1,0,0,-0.492196,0.375419,0,0,0,0,1,0,0,0,0,1,0
1,201,0,0,0,-0.461974,0.375419,0,0,0,0,0,1,0,0,0,0,0
2,202,1,0,0,-0.435580,0.375419,0,0,0,0,0,0,0,0,0,0,1
3,203,0,1,0,2.496407,0.375419,1,0,0,0,0,0,0,1,0,0,0
4,204,0,0,0,-0.480572,0.375419,0,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,0,1,-0.417118,0.375419,0,0,0,0,0,1,0,0,0,0,1
163201,168549,0,0,1,-0.409050,0.375419,1,0,0,0,0,0,0,0,1,0,0
163202,168550,1,0,1,-0.231818,0.375419,0,0,0,0,0,0,1,0,0,0,1
163203,168551,0,1,1,2.496407,0.375419,0,0,0,0,0,1,0,0,0,1,0


,post_id,text_cluster,distance_to_cluster_0,distance_to_cluster_1,distance_to_cluster_2,distance_to_cluster_3,distance_to_cluster_4,distance_to_cluster_5,distance_to_cluster_6,distance_to_cluster_7,...,distance_to_cluster_11,distance_to_cluster_12,distance_to_cluster_13,distance_to_cluster_14,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,9,3.275572,3.443513,3.469100,2.986742,1.948664,3.592508,3.383040,3.612148,...,3.400342,2.830196,2.372013,3.463925,0,0,0,0,0,0
1,2,9,3.192135,3.133694,3.244608,2.846341,2.196497,3.373540,3.331997,3.351419,...,3.336037,2.545622,2.334076,3.385151,0,0,0,0,0,0
2,3,9,2.974873,3.141050,3.392259,3.072776,1.817808,3.338185,3.459793,3.359396,...,3.375446,2.862795,2.404512,3.426456,0,0,0,0,0,0
3,4,4,3.202792,3.771742,4.060500,3.302980,2.426471,3.235161,3.748735,3.805814,...,3.738622,3.358398,2.818986,3.645685,0,0,0,0,0,0
4,5,4,2.978371,2.805885,3.242360,2.653685,1.472535,3.043557,2.781908,3.042039,...,2.817385,2.136082,2.047634,3.083162,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,11,3.082503,1.834552,3.396313,2.826148,3.000543,2.006451,1.805223,2.940562,...,1.308614,2.298392,2.735910,3.154152,0,0,1,0,0,0
7019,7316,11,2.887662,1.800025,3.383409,2.509571,2.963336,1.797055,1.512296,2.583630,...,0.929435,2.204309,2.440939,2.938149,0,0,1,0,0,0
7020,7317,11,2.693023,1.996829,3.464333,2.553049,3.183830,2.234073,2.017127,2.375831,...,1.542911,2.417808,2.809133,2.899021,0,0,1,0,0,0
7021,7318,6,3.359132,1.617927,3.410661,3.112874,3.194130,1.661280,1.049354,3.296159,...,1.469077,2.262688,2.982443,3.463953,0,0,1,0,0,0


,user_id,post_id
0,156604,573
1,142997,6445
2,122594,4554
3,8663,3837
4,94081,5671
...,...,...
99995,137312,1428
99996,130512,1449
99997,95202,5100
99998,2977,6657


,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7314,Although the likeliness of someone focusing on...,movie
7019,7315,"OK, I would not normally watch a Farrelly brot...",movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


## 🎯 8. Выводы и результаты

### Ключевые результаты:
1. **Качество модели (DL)**: AUC-ROC = 0.62-0.65
2. **Качество модели (prod)**: HitRate@5 = 0.604
3. **Среднее время работы одного запроса:** 0,5 секунд